# FNO JAX Tutorial

The JAX tutorial of FNO to learn the 1D Burger's equation is [here](https://github.com/Ceyron/machine-learning-and-simulation/blob/main/english/neural_operators/simple_FNO_in_JAX.ipynb).